In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [5]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
eeg_dataset, splits_path, splits_shuffled_path = os.listdir(torch_models_dir)

In [6]:
eeg_dataset = os.path.join(torch_models_dir, eeg_dataset)
splits_path = os.path.join(torch_models_dir, splits_path)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_path)

/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth 
 /media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth


In [7]:
opt = {
    # Dataset options
    "iv": "image",
    "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "run": "none",
    "eeg_dataset": eeg_dataset,
    "splits_path": splits_path,
    "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"incremental",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.iv)

image


In [8]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from data_loader import EEGDataset, Splitter, SplitterWithData
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

In [9]:
def load_dataset(iv,
             offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-4) - 5 fold cross validation
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):        
    # Load dataset
    dataset = EEGDataset(iv, eeg_dataset, classifier, map_idx = None)
    print("DONE: LOAD DATASET")
    # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
    if kind=="from-scratch":
        relabel = False
    if kind=="incremental":
        relabel = False
    if kind=="no-model-file":
        relabel = True
    splitter = {split: SplitterWithData(iv,
                    dataset,
                    splits_path,
                    classes,
                    split_num,
                    split,
                    relabel) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [12]:
# Stacked BiLSTM network
opt.classifier = "Stacked_BiLSTM"
opt.batch_size = 16
opt.kind = "from-scratch"
opt.run = "imagenet40-1000"
opt.fold = 5

In [14]:
dataset, loaders, splitter = load_dataset(opt.iv,
                             opt.offset,
                             opt.eeg_dataset,
                             opt.splits_path,
                             0, #split_num
                             n_classes,
                             classes,
                             opt.classifier,
                             opt.batch_size,
                             opt.GPUindex,
                             length,
                             channel,
                             min_CNN,
                             opt,
                             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [15]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 32000 idx, val: 4000 idx, test: 4000 idx}
# Each loader batch has {train: 2000 idx, val: 250idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i in range(0, 40):
    eeg, label_val = splitter["val"][i*100]
    eeg, label_train = splitter["train"][i*800]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")
# print(splitter["val"].split_idx[:10])

<class 'data_loader.EEGDataset'>
<class 'dict'>
3 [2000, 250, 250]
1: Label val: 0; label train: 0
2: Label val: 17; label train: 17
3: Label val: 28; label train: 28
4: Label val: 7; label train: 7
5: Label val: 33; label train: 33
6: Label val: 12; label train: 12
7: Label val: 21; label train: 21
8: Label val: 3; label train: 3
9: Label val: 25; label train: 25
10: Label val: 36; label train: 36
11: Label val: 10; label train: 10
12: Label val: 15; label train: 15
13: Label val: 19; label train: 19
14: Label val: 31; label train: 31
15: Label val: 23; label train: 23
16: Label val: 5; label train: 5
17: Label val: 38; label train: 38
18: Label val: 8; label train: 8
19: Label val: 1; label train: 1
20: Label val: 34; label train: 34
21: Label val: 29; label train: 29
22: Label val: 26; label train: 26
23: Label val: 13; label train: 13
24: Label val: 11; label train: 11
25: Label val: 22; label train: 22
26: Label val: 18; label train: 18
27: Label val: 6; label train: 6
28: Label v

In [13]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_Stacked_BiLSTM(
  (stacked_bilstm): LSTM(96, 128, num_layers=2, batch_first=True, bidirectional=True)
  (output1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_Stacked_BiLSTM                [1, 40]                   --
├─LSTM: 1-1                              [1, 512, 256]             626,688
├─Linear: 1-2                            [1, 128]                  32,896
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 664,744
Trainable params: 664,744
Non-trainable params: 0
Total mult-adds (M): 320.90
Input size (MB): 0.20
Forward/backward pass size (MB): 1.05
Params size (MB): 2.66
Estimated Total Size (MB): 3.91

In [16]:
model_path = (opt.iv+
                  "-"+
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel)+
                  "-"+
                  str(0)) + "-noshuffle"
                  
channel_idx=None
print(model_path)

image-Stacked_BiLSTM-512-96-0-noshuffle


In [17]:
print(opt)

Namespace(iv='image', offset=None, results_file='results.pkl', subject=0, run='imagenet40-1000', eeg_dataset='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth', splits_path='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth', fold=5, batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='Stacked_BiLSTM')


In [18]:
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.701542615890503; accuracy=0.021437501534819603
Batch 2000: Loss=3.7104787826538086; accuracy=0.022093750536441803
Validation accuracy: 0.0247500017285347
epoch 2
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.686978340148926; accuracy=0.024812500923871994
Batch 2000: Loss=3.6926369667053223; accuracy=0.024281250312924385
Validation accuracy: 0.02500000037252903
epoch 3
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.691617727279663; accuracy=0.02562500163912773
Batch 2000: Loss=3.7041683197021484; accuracy=0.02459375187754631
Validation accuracy: 0.027250001206994057
epoch 4
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([1

Validation accuracy: 0.030000001192092896
epoch 29
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=0.8207628726959229; accuracy=0.6750625371932983
Batch 2000: Loss=1.155935287475586; accuracy=0.6568125486373901
Validation accuracy: 0.026750002056360245
epoch 30
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=1.1611584424972534; accuracy=0.6928750276565552
Batch 2000: Loss=0.676348865032196; accuracy=0.6739687919616699
Validation accuracy: 0.02800000086426735
epoch 31
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=2.15395450592041; accuracy=0.7117500305175781
Batch 2000: Loss=0.9057379961013794; accuracy=0.6940625309944153
Validation accuracy: 0.030000001192092896
epoch 32
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size

Validation accuracy: 0.02850000187754631
epoch 57
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=0.9046136140823364; accuracy=0.8820000290870667
Batch 2000: Loss=0.8923779726028442; accuracy=0.8634375333786011
Validation accuracy: 0.026500001549720764
epoch 58
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=0.19030234217643738; accuracy=0.8735000491142273
Batch 2000: Loss=0.5411105155944824; accuracy=0.8636875152587891
Validation accuracy: 0.026000000536441803
epoch 59
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=0.56320720911026; accuracy=0.8861875534057617
Batch 2000: Loss=1.3177130222320557; accuracy=0.8733437657356262
Validation accuracy: 0.027500001713633537
epoch 60
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.S

Validation accuracy: 0.030250001698732376
epoch 85
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=1.1378132104873657; accuracy=0.9161250591278076
Batch 2000: Loss=0.2987479269504547; accuracy=0.9023438096046448
Validation accuracy: 0.026250001043081284
epoch 86
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=0.40077680349349976; accuracy=0.908875048160553
Batch 2000: Loss=0.3096345365047455; accuracy=0.8939688205718994
Validation accuracy: 0.026500001549720764
epoch 87
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=0.10238569974899292; accuracy=0.9105625152587891
Batch 2000: Loss=0.7748597860336304; accuracy=0.8992187976837158
Validation accuracy: 0.02525000087916851
epoch 88
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch

In [19]:
val =accuracy_val
test = accuracy_test

print("Validation accuracy: ", val)
print("Test accuracy: ", test)

Validation accuracy:  0.024000000208616257
Test accuracy:  0


In [ ]:
print(accuracy_val)